In [9]:
# handling data, Refactored code from https://github.com/Coopss/EMNIST/blob/master/training.py

In [20]:
import numpy as np

from scipy.io import loadmat

import pickle

In [21]:
def load_data(mat_file_path, width=28, height=28, max_data=None, verbose=True):
    ''' Load data in from .mat file as specified by the paper.
        Arguments:
            mat_file_path: path to the .mat, should be in sample/
        Optional Arguments:
            width: specified width
            height: specified height
            max_: the max number of samples to load
            verbose: enable verbose printing
        Returns:
            A tuple of training and test data, and the mapping for class code to ascii value,
            in the following format:
                - ((training_images, training_labels), (testing_images, testing_labels), mapping)
    '''
    # Load convoluted list structure form loadmat
    mat = loadmat(mat_file_path)
    
    # Load char mapping
    mapping = {kv[0]:kv[1:][0] for kv in mat['dataset'[0][0][2]]}
    pickle.dump(mapping, open('bin/mapping.p', 'wb'))
    
    training_image_row = mat['dataset'][0][0][0][0][0][0];
    training_label_row = mat['dataset'][0][0][0][0][0][1];
    
    testing_image_row = mat['dataset'][0][0][1][0][0][0];
    testing_label_row = mat['dataset'][0][0][1][0][0][1];
    
    # Load training data
    max_training_data = len(training_image_row) if max_data == None else max_data
    
    training_images = training_image_row[:max_training_data]
    training_labels = training_label_row[:max_training_data]
    
    # Load testing data, use 1/6 of max_data if provided
    max_testing_data = len(testing_image_row) if max_data == None else max_data/6
    
    testing_images = testing_image_row[:max_testing_data]
    testing_labels = testing_image_row[:max_testing_data]
    
    # Reshape training data
    training_size = len(training_images)
    for i in range(training_size):
        if verbose: print('%d/%d (%.2lf%%)' % (i + 1, training_size, ((i + 1)/training_size) * 100), end='\r')
        training_images[i] = reshape(training_images[i])
    if verbose: print('')
        
    # Reshape testing data
    testing_size = len(testing_images)
    for i in range(testing_size):
        if verbose: print('%d/%d (%.2lf%%)' % (i + 1, testing_size, ((i + 1)/testing_size) * 100), end='\r')
        testing_size[i] = reshape(testing_size[i])
    if verbose: print('')
    
    # Extend the arrays to (None, 28, 28, 1)
    training_images = training_images.reshape(training_images.shape[0], height, width, 1)
    testing_images = testing_images.reshape(testing_images.shape[0], height, width, 1)
    
    # Convert type to float32
    training_images = training_images.astype('float32')
    testing_images = testing_images.astype('float32')

    # Normalize to prevent issues with model
    training_images /= 255
    testing_images /= 255

    return ((training_images, training_labels), (testing_images, testing_labels), mapping)
        

In [22]:
def reshape(img):
    # Used to rotate images (for some reason they are transposed on read-in)
    img.shape = (width, height)
    img = img.T
    img = list(img)
    img = [item for sublist in img for item in sublist]
    return img

def display(img, threshold=0.5):
    # Debugging
    render=''
    for row in img:
        for col in row:
            if col > threshold: 
                render += '@'
            else: 
                render += '.'
        render += '\n'
    return render